<a href="https://colab.research.google.com/github/vadManuel/Machine-Learning-UCF/blob/hw1_single_plot/Homework/hw1/mvasquez_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [0]:
#@title Critical point check [double click to open] { display-mode: "form" }
def get_H(func_xx, func_yy, func_xy, x, y):
  return func_xx(x,y)*func_yy(x,y)-func_xy(x,y)**2

def get_critical_point_type(func_xx, func_yy, func_xy, x, y):
  if get_H(func_xx, func_yy, func_xy, x, y) > 0:  # H > 0 ::= max/min
    if func_xx(x, y) > 0: # func_xx(x, y) > 0 ::= min
      return 'min'
    if func_xx(x, y) < 0: # func_xx(x, y) < 0 ::= max
      return 'max'
    return 'unknown'      # func_xx(x, y) = 0 ::= unknown
  if get_H(func_xx, func_yy, func_xy, x, y) < 0:  # H < 0 ::= saddle point
    return 'saddle point'
  return 'unknown'                                # H = 0 ::= unknown

In [0]:
#@title Functions (min,max,saddle) [double click to open] { display-mode: "form" }
def f(x, y):
  return (x+3)**2+y**2
def f_xx(x, y):
  return 2
def f_yy(x, y):
  return 2
def f_xy(x, y):
  return 0

def g(x,y):
  return -(x**2-1)**2-(x**2*y-x-1)**2
def g_xx(x,y):
  return -12*x**2*(y**2+1)+12*x*y+4*y+2
def g_yy(x,y):
  return -2*x**4
def g_xy(x,y):
  return 2*x*(-4*x**2*y+3*x+2)

def h(x,y):
  return x**2-y**2
def h_xx(x,y):
  return 2
def h_yy(x,y):
  return -2
def h_xy(x,y):
  return 0

In [0]:
#@title Surface maker [double click to open] { display-mode: "form" }
def make_surface(func, x_range, y_range, critical_points=None, stepping=.005):
  xx = np.linspace(x_range[0], x_range[1])
  yy = np.linspace(y_range[0], y_range[1])
  x,y = np.meshgrid(xx,yy)
  v_func = np.vectorize(func)
  z = v_func(x,y)

  x_scale = abs(x_range[0])+abs(x_range[1])
  y_scale = abs(y_range[0])+abs(y_range[1])
  z_min = np.min([x,y,z])
  z_max = np.max([x,y,z])
  z_scale = abs(z_min)+abs(z_max)

  data = [
    go.Surface(
      x=x,
      y=y,
      z=z,
      colorscale='rainbow',
      opacity=1,
      name=''
    ),
    go.Surface(
      x=np.min(xx)*np.ones(z.shape)-x_scale*.15,
      y=y,
      z=z,
      colorscale='gray',
      opacity=.8,
      showscale=False,
      surfacecolor=z, # mapping depth
      name='yz-projection'
    ),
    go.Surface(
      x=x,
      y=np.min(yy)*np.ones(z.shape)-y_scale*.15,
      z=z,
      colorscale='gray',
      opacity=.8,
      showscale=False,
      surfacecolor=z, # mapping depth
      name='xz-projection'
    ),
    go.Surface(
      x=x,
      y=y,
      z=np.min(z)*np.ones(z.shape)-z_scale*.15,
      colorscale='gray',
      opacity=.7,
      showscale=False,
      surfacecolor=z, # mapping depth
      name='xy-projection'
    )
  ]

  contours = [go.Contour(
      x=xx,
      y=yy,
      z=z,
      colorscale='rainbow',
      showscale=False,
      contours=dict(
          start=z_min,
          end=z_max,
          size=(abs(z_min)+abs(z_max))*stepping
      )
  )]

  if critical_points:
    theta = np.linspace(0,2*np.pi)
    phi = np.linspace(0,np.pi)
    x = np.outer(np.cos(theta), np.sin(phi))*x_scale*.02  # reshape to 3D
    y = np.outer(np.sin(theta), np.sin(phi))*y_scale*.02  # reshape to 3D
    z = np.outer(np.ones(len(theta)), np.cos(phi))*z_scale*.015 # reshape to 3D

    for i, critical_point in enumerate(critical_points):
      data.append(go.Surface(
          x=x+critical_point[0],
          y=y+critical_point[1],
          z=z,
          colorscale='gray',
          showscale=False,
          name=('zero #%d' % (i+1))
      ))
      contours.append(go.Scatter(
          x=[critical_point[0]],
          y=[critical_point[1]],
          line=dict(color='black'),
          showlegend=False,
          name=('zero #%d' % (i+1))
      ))
  
  return [z_min, z_max], data, contours

In [0]:
#@title Display plots [double click to open] { display-mode: "form" }
def show_fig(surfaces, contours, z_range, title=None, stepping=.005):
  cols = 2
  specs = [[{'type': 'surface'},{'type': 'contour'}]]
  fig = make_subplots(rows=1, cols=cols, specs=specs)

  for surface in surfaces:
    fig.add_trace(surface, row=1, col=1)
  start, end, size = (np.floor(z_range[0]), np.ceil(z_range[1]), (abs(z_range[0])+abs(z_range[1]))*stepping)
  fig.update_traces(contours_z=dict(show=True, start=start, end=end, size=size, color='steelblue',
                                    project_x=True, project_y=True, project_z=True))
  for contour in contours:
    fig.add_trace(contour, row=1, col=2)

  fig.update_layout(title=title, autosize=False,
                    scene_camera_eye=dict(x=-1, y=-2, z=1.5),
                    margin=dict(l=10, r=0, b=10, t=40), height=400)

  fig.show()

## Poblem 1
Find three simple functions from R^2 to R and suitable points that are:
- a local minimum
- a local maximum
- neither a minimum nor maximum, but the gradient is zero at this point.

Plot these functions using 3D plots as in the notebooks for creating density and contour plots and three dimensional plots. Indicate clearly the points. Use 2d and 3d contour plots.

In [6]:
stepping = .025
z_range, surfaces, contours = make_surface(f, [-10,10], [-10,10], critical_points=[[-3,0]], stepping=stepping)
show_fig(surfaces, contours, z_range, stepping=stepping, title='[P1] local minimum | f(x,y)=(x+3)^2+y^2')

In [7]:
z_range, surfaces, contours = make_surface(g, [-2,2], [-2,6], critical_points=[[-1,0],[1,2]])
show_fig(surfaces, contours, z_range, title='[P1] local maximum | f(x,y)=-(x^2-1)^2-(yx^2-x-1)^2')

In [8]:
z_range, surfaces, contours = make_surface(h, [-2,2], [-2,6], critical_points=[[0,0]], stepping=stepping)
show_fig(surfaces, contours, z_range, stepping=stepping, title='[P1] saddle point | f(x,y)=x^2-y^2')